# Import & Dataset 읽어오기

In [1]:
from datasets import concatenate_datasets, load_from_disk
from src.retriever.retrieval.sparse_retrieval import SparseRetrieval
from transformers import AutoTokenizer

org_dataset = load_from_disk('./data/train_dataset')
print(org_dataset)
full_ds = concatenate_datasets(
        [
            org_dataset["train"].flatten_indices(),
            org_dataset["validation"].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level2-mrc-nlp-07\level2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


# Retriever 선언 및 Retriever 진행
* 아마 초기엔 학습 진행하는데 시간 좀 걸릴예정
    * 약 15분정도 소요(승범 컴 기준)
    * 노션에 mrc/검증결과/Sparse 처리에 가장 좋은 bm25올려났으므로 그거 data 폴더 안에 넣으면 실행가능.

In [6]:
# 위에서 선언한거 가져오기 Retriever
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=False,)
retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path="./data/",
    context_path="wikipedia_documents.json",
    mode = "bm25",
    max_feature=1000000,
    ngram_range=(1,2),
    #tokenized_docs = tokenized_docs,
)
retriever.get_sparse_embedding()

# Top k 조절.
df = retriever.retrieve(full_ds, topk=10)

# 여기서 df가 reader가 사용할 데이터 프레임입니다.
df.head(5)

Lengths of unique contexts : 56737
Building bm25 embedding...
Start Initializing...


Tokenizing...: 100%|██████████| 56737/56737 [01:16<00:00, 740.93it/s]


Generating n-grams and building vocabulary...


Generating n-grams: 100%|██████████| 56737/56737 [00:21<00:00, 2658.83it/s]


Current mode : bm25
End Initialization


Calculating BM25: 100%|██████████| 57/57 [12:42<00:00, 13.38s/it]


Finish BM25 Embedding
bm25 embedding shape: (56737, 1000000)
(4192, 1000000) (56737, 1000000)
result shape : (4192, 56737)
[query exhaustive search] done in 41.138 s


Sparse retrieval: 100%|██████████| 4192/4192 [00:00<00:00, 4668.52it/s] 


,question,id,retrieval_context,original_context,answers
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 ...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}"
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}"
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"1943년 초, 마리아 발토르타가 9년 간 건강 상태가 안 좋아졌을 때, 그녀의 고...",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}"
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,이 불상군은 1978년 발견되어 학계에 알려졌으며 봉황리 햇골산 중턱 두 곳에 약간...,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}"
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,양산 통도사의 금동천문도의 전면에는 천구(天球)의 북극을 중심으로 둥글게 북극으로 ...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}"


* question : 질문
* id : mrc 번호
* retrieval_context : retrieval 결과로 나온 topk를 하나의 text로 만든것
* original_context : 실제 retrieval 정답
* answers: dict 형태
    * answers_start : original_context에서 어디에서 시작하는지
    * text: 정답의 text값이 뭔지 

In [8]:
len(df['retrieval_context'][0])

13059